In [ ]:
#| default_exp atom_finance

In [3]:
#| export
import os

import json
import requests

# dev mode has asset limitations 
# https://docs.atom.finance/reference/authentication
DEV_KEY_EQUITIES = ["AAPL", "BABA", "META", "JPM", "NKE", "VALE3", "PETR3", "RY", "SHOP"]
DEV_KEY_FUNDS = ["SPY", "VTSAX"]
DEV_KEY_CRYPTO = ["BTC", "DOGE"]

class AtomFinanceClient:
    def __init__(self, mode = "prod"):
        # Creds
        self.api_key = os.getenv("ATOM_FINANCE_API_KEY")

        # Endpoints
        if mode == "prod":
            self.endpoint = "https://platform.atom.finance/api/2.0/{path}?api_key={api_key}"
        elif mode == "dev":
            self.endpoint = "https://sandbox-platform.atom.finance/api/2.0/{path}?api_key={api_key}"

        # Headers
        self.headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }

    def post(self, path: str = "search", data: dict = {}):
        payload = json.dumps(data)
        endpoint = self.endpoint.format(**{
            "path" : path,
            "api_key" : self.api_key
                }
            )

        resp = requests.post(
                endpoint,
                data = payload,
                headers = self.headers
                )
        if resp.status_code == 201:
            return resp.json()
        else:
            print(resp.json())
            raise Exception(f"POST to {path} failed")

    def equity_overview(self, symbol):
        template = {
              "asset": {
                "identifier": "ticker",
                "value": symbol,
                "assetType": "equity",
                "market": "USA"
              }
            }
        return self.post(path = "equity/overview", data = template)

    def test_authenticate(self):
        resp = self.post(data = {"query" : "cars"})
        print("Test authentication to Atom successful")


In [5]:
atom = AtomFinanceClient(mode = "dev")
atom.test_authenticate()
equity_overview = atom.equity_overview(symbol = "AAPL")
print("atom.equity_overview(symbol = 'AAPL')")
print(equity_overview)

{'error': {'type': 'unauthorized_access', 'message': 'Invalid api_key'}}


Exception: POST to search failed